In [32]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import metrics

In [33]:
data_csv = pd.read_csv('EURUSD_data_D1_change.csv')

In [34]:
data_csv.drop(['Unnamed: 0'], axis=1, inplace=True)

In [35]:
print(data_csv)

        close  change
0     1.16680   -5.90
1     1.17140    4.60
2     1.15990  -11.50
3     1.14230  -17.60
4     1.14040   -1.90
...       ...     ...
6005  1.08319    3.90
6006  1.08386    0.67
6007  1.08060   -3.26
6008  1.08469    4.09
6009  1.08173   -2.96

[6010 rows x 2 columns]


In [36]:
# #Max-Min标准化
# #建立MinMaxScaler对象
scaler = preprocessing.StandardScaler()
# # 标准化处理
data_origin = np.array(data_csv)
data = scaler.fit_transform(data_origin)

data

array([[-0.17168076, -0.80101863],
       [-0.14297585,  0.62818963],
       [-0.21473812, -1.56326304],
       ...,
       [-0.70958581, -0.44167484],
       [-0.6840634 ,  0.55877095],
       [-0.70253439, -0.40084032]])

In [6]:
# n = 0
# for i in range(data.shape[0]):
#     if data[i,0]<0:
#         n=n+1
# print(n)

In [37]:
data_y = data[-6000:,:]
print(data_y.shape)

(6000, 2)


In [38]:
data_x = []
for r in range(6000):
    for c in range(10):
        item = []
        item.append(data[r+c, 0])
        item.append(data[r+c, 1])
        data_x.append(item)
data_x = np.array(data_x)
print(data_x)
print(data_x.shape)

[[-0.17168076 -0.80101863]
 [-0.14297585  0.62818963]
 [-0.21473812 -1.56326304]
 ...
 [-0.68924277  0.0932574 ]
 [-0.70958581 -0.44167484]
 [-0.6840634   0.55877095]]
(60000, 2)


In [39]:
data_x=data_x.reshape(6000,10,2)

In [40]:
data_x.shape

(6000, 10, 2)

In [41]:
data_x

array([[[-0.17168076, -0.80101863],
        [-0.14297585,  0.62818963],
        [-0.21473812, -1.56326304],
        ...,
        [-0.36325483, -0.29739286],
        [-0.40256808, -0.85546466],
        [-0.38259944,  0.43762853]],

       [[-0.14297585,  0.62818963],
        [-0.21473812, -1.56326304],
        [-0.32456561, -2.39356498],
        ...,
        [-0.40256808, -0.85546466],
        [-0.38259944,  0.43762853],
        [-0.39445582, -0.25655834]],

       [[-0.21473812, -1.56326304],
        [-0.32456561, -2.39356498],
        [-0.33642198, -0.25655834],
        ...,
        [-0.38259944,  0.43762853],
        [-0.39445582, -0.25655834],
        [-0.37885533,  0.34234798]],

       ...,

       [[-0.69585738,  0.05106172],
        [-0.70197277, -0.13133247],
        [-0.66790129,  0.7452486 ],
        ...,
        [-0.71776047, -0.62543019],
        [-0.6934237 ,  0.53290908],
        [-0.68924277,  0.0932574 ]],

       [[-0.70197277, -0.13133247],
        [-0.66790129,  0.74

In [42]:
data_y.shape

(6000, 2)

In [43]:
data_x_train = data_x[:5000,:, :]
data_y_train = data_y[:5000,:]

data_x_test = data_x[-1000:,:, :]
data_y_test = data_y[-1000:,:]

In [44]:
print(data_x_train.shape)
print(data_y_train.shape)
print(data_x_test.shape)
print(data_y_test.shape)

(5000, 10, 2)
(5000, 2)
(1000, 10, 2)
(1000, 2)


In [15]:

#     model = Sequential()
 
#     model.add(LSTM(input_dim=10,output_dim=128,return_sequences=True))
#     model.add(Dropout(0.1))
 
#     model.add(LSTM(128,return_sequences=False))
#     model.add(Dropout(0.1))
 
#     model.add(Dense(output_dim=1))
#     model.add(Activation("linear"))
 
#     model.compile(loss="mse", optimizer="rmsprop")

In [16]:
# model = Sequential()

# model.add(LSTM(output_dim=128, 
#                 input_shape=(10, 1),
#                 activation='relu',
#                 return_sequences=True))
# for i in range(6):
#     model.add(LSTM(output_dim=128,
#                     activation='tanh',
#                     return_sequences=True))
# #     model.add(Dropout(0.1))

# model.add(LSTM(output_dim=1,
#                activation='linear',
#                return_sequences=False))

# model.compile(loss='mse', optimizer='adam', metrics=[metrics.mse])
# model.summary()

In [45]:
def create_model(output_dim1, num_layer, output_dim2):
    model = Sequential()

    model.add(LSTM(output_dim=output_dim1, 
                input_shape=(10, 2),
                activation='relu',
                return_sequences=True))
    for i in range(num_layer):
        model.add(LSTM(output_dim=output_dim2,
                        activation='tanh',
                        return_sequences=True))
#     model.add(Dropout(0.1))
    model.add(LSTM(output_dim=output_dim2,
                        activation='tanh'))
    model.add(Dense(2))

    model.compile(loss='mse', optimizer='adam', metrics=[metrics.mse])
#     model.summary()
    return model

In [18]:
# from sklearn.model_selection import RandomizedSearchCV
# from skopt.space import Real, Categorical, Integer
# from keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.model_selection import train_test_split

In [19]:
# #采用暴力搜索，所有参数进行组合，然后选择最好的参数
# tuned_parameters=[{'output_dim1':Integer(28, 128),
#                    'num_layer':Integer(3,12),
#                    'output_dim2':Integer(28, 128)}]
# # 'output_dim1':Integer(28, 128),
# #             'num_layer':Integer(3,12),
# #             'output_dim2':Integer(28, 128)

In [20]:
# train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, train_size=0.80, random_state=0)

In [21]:
# model=KerasRegressor(build_fn=create_model)

In [22]:
# #采用随机搜索，给参数一个范围，然后系统随机选择参数，进行检验，然后选择最好的
# tuned_parameters={'output_dim1':[28, 64, 96, 128],
#                    'num_layer':[4,5,6],
#                    'output_dim2':[28, 64, 96, 128]}

# lstm=RandomizedSearchCV(model,tuned_parameters,scoring='neg_mean_squared_error',n_iter=2)
 
# lstm.fit(data_x_train,data_y_train)
# print('best parameters:',lstm.best_estimator_)
# # print(classification_report(test_y,clf.predict(test_x)))
# # print(metrics.confusion_matrix(test_y,clf.predict(test_x)))

In [23]:
# import sklearn
# sorted(sklearn.metrics.SCORERS.keys())

In [24]:
# from sklearn.model_selection import train_test_split
# from skopt import BayesSearchCV
# from skopt.space import Real, Categorical, Integer

In [25]:
# class FixedBayesSearchCV(BayesSearchCV):
#     """
#     Dirty hack to avoid compatibility issues with sklearn 0.2 and skopt.
#     Credit: https://www.kaggle.com/c/home-credit-default-risk/discussion/64004
#     For context, on why the workaround see:
#         - https://github.com/scikit-optimize/scikit-optimize/issues/718
#         - https://github.com/scikit-optimize/scikit-optimize/issues/762
#     """
#     def __init__(self, estimator, search_spaces, optimizer_kwargs=None,
#                 n_iter=50, scoring=None, fit_params=None, n_jobs=1,
#                 n_points=1, iid=True, refit=True, cv=None, verbose=0,
#                 pre_dispatch='2*n_jobs', random_state=None,
#                 error_score='raise', return_train_score=False):
#         """
#         See: https://github.com/scikit-optimize/scikit-optimize/issues/762#issuecomment-493689266
#         """
 
#         # Bug fix: Added this line
#         self.fit_params = fit_params
 
#         self.search_spaces = search_spaces
#         self.n_iter = n_iter
#         self.n_points = n_points
#         self.random_state = random_state
#         self.optimizer_kwargs = optimizer_kwargs
#         self._check_search_space(self.search_spaces)
 
#         # Removed the passing of fit_params to the parent class.
#         super(BayesSearchCV, self).__init__(
#                 estimator=estimator, scoring=scoring, n_jobs=n_jobs, iid=iid,
#                 refit=refit, cv=cv, verbose=verbose, pre_dispatch=pre_dispatch,
#                 error_score=error_score, return_train_score=return_train_score)
 
#     def _run_search(self, x):
#         raise BaseException('Use newer skopt')

In [26]:
# def lstm_auto_para_tuning_bayesian(model,data_x,data_y):
#     train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, train_size=0.80, random_state=0)
#     # cv：交叉验证 n_points：并行采样的超参组数
#     opt = FixedBayesSearchCV(model,cv=5,n_points=2,n_jobs=4,verbose=1,
#         search_spaces={
#             'output_dim1':Integer(28, 128),
#             'num_layer':Integer(3,12),
#             'output_dim2':Integer(28, 128)
#         },
#          fit_params={
#                  'eval_set':[(test_x, test_y)],
#                  'eval_metric': 'mse',
#                  'early_stopping_rounds': 50
#                  })
#     opt.fit(train_x,train_y)
#     print("val. score: %s" % opt.best_score_)
#     print("test score: %s" % opt.score(test_x, test_y))
#     print("Best parameters: ", opt.best_params_)
#     print("Best estimator:", opt.best_estimator_)

In [46]:
model = create_model(128, 8, 128)

e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(10, 2), activation="relu", return_sequences=True, units=128)`
  import sys
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, units=128)`
  # This is added back by InteractiveShellApp.init_path()
e:\envs\python36env\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", units=128)`
  


In [28]:
# lstm_auto_para_tuning_bayesian(model,data_x,data_y)

In [47]:
model.fit(data_x_train, data_y_train, epochs=20, batch_size=2000, verbose=1)

Epoch 1/20
5000/5000 [==============================] - 60s 12ms/step - loss: 1.1319 - mean_squared_error: 1.1319
Epoch 2/20
5000/5000 [==============================] - 32s 6ms/step - loss: 1.1302 - mean_squared_error: 1.1302
Epoch 3/20
5000/5000 [==============================] - 32s 6ms/step - loss: 1.1296 - mean_squared_error: 1.1296
Epoch 4/20
5000/5000 [==============================] - 32s 6ms/step - loss: 1.1253 - mean_squared_error: 1.1253
Epoch 5/20
5000/5000 [==============================] - 33s 7ms/step - loss: 1.0912 - mean_squared_error: 1.0912
Epoch 6/20
5000/5000 [==============================] - 34s 7ms/step - loss: 0.8591 - mean_squared_error: 0.8591
Epoch 7/20
5000/5000 [==============================] - 34s 7ms/step - loss: 0.6890 - mean_squared_error: 0.6890
Epoch 8/20
5000/5000 [==============================] - 34s 7ms/step - loss: 0.6101 - mean_squared_error: 0.6101
Epoch 9/20
5000/5000 [==============================] - 34s 7ms/step - loss: 0.6658 - mean_squa

In [48]:
# 评估模型,不输出预测结果
loss,accuracy = model.evaluate(data_x_test,data_y_test)
print('\ntest loss','{:.5f}'.format(loss))
print('accuracy','{:.5f}'.format(accuracy))

1000/1000 [==============================] - 16s 16ms/step

test loss 0.23667
accuracy 0.23667


In [49]:
y_pred = model.predict(data_x_test,batch_size = 1)

In [32]:
# y_pred[-10:]

In [33]:
# data_y_test[-10:,0]

In [34]:
# data_x_test[-10:,0,-1]

In [51]:
t = []
for i in range(1000):
    if data_y_test[:,0][i]>data_x_test[:,0,-1][i]:
        t.append(1)
    else:
        t.append(0)
len(t)

1000

In [52]:
p = []
for i in range(1000):
    if y_pred[:,0][i]>data_x_test[:,0,-1][i]:
        p.append(1)
    else:
        p.append(0)
len(p)

1000

In [54]:
n = 0
for i in range(1000):
    if t[i]==p[i]:
        n=n+1
n/1000

0.956

In [38]:
# model.save('model.h5')#保存名为model的h5文件到程序所在目录

In [39]:
# data_x_test.shape

In [40]:
# model.predict(data_x_test,batch_size = 1)

In [41]:
# x_pred = [1.08244, 1.0776, 1.08229, 1.08564, 1.08607, 1.08739, 1.08386, 1.09066, 1.09798, 1.0912]
# 

In [42]:
# x_pred = np.array(x_pred).reshape(1,10,1)

In [43]:
# x_pred.shape

In [44]:
# model.predict(x_pred,batch_size = 1).tolist()[0][0]

In [73]:
y_pred.shape

(2000, 1)

In [55]:
y_pred

array([[-0.5610287 ,  0.04343047],
       [-0.6556678 ,  0.04473378],
       [-0.78354853,  0.04618564],
       ...,
       [-0.840467  ,  0.04614529],
       [-0.80089027,  0.04593469],
       [-0.7523852 ,  0.04550268]], dtype=float32)

In [78]:
y_t = data_y_test.reshape(2000)

In [79]:
y_p = y_pred.reshape(2000)

In [76]:
 df_test_output = pd.DataFrame(columns=['真实值', '预测值'])

In [80]:
df_test_output['真实值'] = y_t

In [81]:
df_test_output['预测值'] = y_p

In [82]:
df_test_output

,真实值,预测值
0,1.22287,1.215008
1,1.21822,1.212196
2,1.23848,1.209206
3,1.23980,1.209921
4,1.23972,1.212526
...,...,...
1995,1.09347,1.107315
1996,1.09120,1.101921
1997,1.09798,1.098248
1998,1.09066,1.093867


In [83]:
df_test_output.to_csv('EURUSD_test_output.csv',encoding='utf_8_sig')